In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import pandas as pd

# 資料夾路徑
folder_path = r"C:\Users\yankee\Desktop\研究\程式\維大力3"
folder_path1= r"C:\Users\yankee\Desktop\研究\程式\維大力除造點3"

# 初始化空列表，用於存儲結果
results = []
# 初始化空列表，用於存儲 max_x_difference
max_x_difference_list = []
#初始化空列表，用於存儲speed
speed_list=[]
#初始化空列表，用來儲存opened_image
opened_images=[]
#設定範圍
pixal_range=11
#FPS值
FPS=60
t=1/60
#幀數
frame_count=0

#計算轉換比例尺
d=33
I=5.1851852*d

# 遍歷資料夾中的所有檔案
for filename in os.listdir(folder_path):
    # 確保檔案是圖像檔（這裡假設所有的檔案都是圖像檔）
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # 讀取圖像
        image_path = os.path.join(folder_path, filename)
        original_image = plt.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # 執行開運算
        kernel = np.ones((3, 3), np.uint8)
        opened_image = cv2.morphologyEx(original_image, cv2.MORPH_OPEN, kernel)

        # 將圖像名稱添加到檔案路徑中
        #output_image_path = os.path.join(folder_path1, f"processed_{filename}")
        # 將圖像儲存到檔案中
        #plt.imsave(output_image_path, opened_image,cmap='gray')

        # 將開運算後的圖像添加到列表中
        opened_images.append(opened_image)

        # 在開運算後的圖像上尋找接近白色的像素
        white_pixels_opened = np.column_stack(np.where(opened_image >= 250))
        
        # 根據 x 座標排序（列索引），這將使像素從左到右排列
        white_pixels_opened_sorted = white_pixels_opened[np.argsort(white_pixels_opened[:, 1])]

        # 過濾在 x = 0~x = 10 的白色像素
        white_pixels_opened_list = [white_pixels_opened_sorted[white_pixels_opened_sorted[:, 1] == i] for i in range(pixal_range)]

        #計算幀數
        frame_count+=1

        # 找到圖像中的最高與最低 y 座標
        for i in range(pixal_range):
         if white_pixels_opened.size>2500:
           if white_pixels_opened_list[i].size > 0 :
             max_y = np.max(white_pixels_opened_list[i][:, 0])
             min_y_sort=(white_pixels_opened_list[i][:, 0][max_y-white_pixels_opened_list[i][:, 0] <= 230])
             min_y=np.min(min_y_sort)
          # 找到圖像中的最遠 x 座標
             max_x = np.max(white_pixels_opened[:, 1])
          # 計算寬度
             y_difference = (max_y - min_y)/I
          # 計算時間
             time=frame_count/FPS
          # 將結果添加到列表中
             results.append({
              'filename': filename,
              'max_x': max_x,
              'y_difference': y_difference,
              'time':time
              }) 
             break 
           else:
             max_x=0
             y_difference=0
             results.append({
             'filename': filename,
             'max_x': max_x,
             'y_difference': y_difference,
             'time':time
             }) 
             break
        
# 顯示結果
for result in results:
    print(f"檔案名稱: {result['filename']}")
    print(f"最遠 x 座標: {result['max_x']}")
    print(f"寬度: {result['y_difference']}")
    print(f"時間:{result['time']}")
    print("===")

# 將 results 清單轉換為 DataFrame
df = pd.DataFrame(results)

# 將 DataFrame 匯出為 Excel 檔案
#df.to_excel('results.xlsx', index=False, engine='openpyxl')

##########################################################

# 遍歷每個字典的索引
i=0
total=0
current_speed=0
while i<(len(results)-1):
    #取出相鄰兩個元素的 'max_x' 值
    max_x_value_1 = results[i]['max_x']
    max_x_value_2 = results[i+1]['max_x']
    #取得寬度
    di=(results[i+1]['y_difference'])
    #時間變數更新
    i1=i
    if max_x_value_1 == 0:
       i+=1
       continue
    elif max_x_value_1 != 0:
     #先印第一個檔案的值
     print(f"i:{i}")
     print(f"檔案名稱1: {results[i]['filename']}, 最遠 x 座標1: {max_x_value_1}")
     if max_x_value_2 == 0:
       while max_x_value_2 == 0:
         i+=1
         if i>=len(results)-1:
           break
         max_x_value_2 = results[i+1]['max_x']  
       if i>=len(results)-1:
           break
       max_x_difference = (max_x_value_2 - max_x_value_1)/I
       i2=i+1
        #印出剩下的值
       print(f"檔案名稱2: {results[i+1]['filename']}, 最遠 x 座標2: {max_x_value_2}")
       print(f"增加的位移量: {max_x_difference}")
     elif max_x_value_2 != 0:
       max_x_difference = (max_x_value_2 - max_x_value_1)/I
       #印出剩下的值
       print(f"檔案名稱2: {results[i+1]['filename']}, 最遠 x 座標2: {max_x_value_2}")
       print(f"增加的位移量: {max_x_difference}")
       i+=1
       i2=i

    # 將 max_x_difference 的值添加到列表中
    max_x_difference_list.append(max_x_difference)
    #計算時間差
    time_difference=(results[i2]['time']-results[i1]['time'])
    #計算速度並加總
    speed=max_x_difference/time_difference
    current_speed+=speed
    #計算加總
    add=((0.5*di)**2)*current_speed
    #計算虛擬體積前進行加總
    total+=add
    
    speed_list.append(current_speed)

    print(f"time:{time_difference}")
    print(f"di:{di}")
    print(f"增加的速度:{speed}")
    print(f"目前的速度:{current_speed}")
    print("===")
#計算體積
volume=math.pi*t*total

print(f"體積:{volume}")    
#print(len(max_x_difference_list))

檔案名稱: frame_0129.jpg
最遠 x 座標: 46
寬度: 0.5785714269183673
時間:2.1666666666666665
===
檔案名稱: frame_0130.jpg
最遠 x 座標: 173
寬度: 0.017532467482374768
時間:2.183333333333333
===
檔案名稱: frame_0131.jpg
最遠 x 座標: 279
寬度: 0.09935064906679035
時間:2.2
===
檔案名稱: frame_0132.jpg
最遠 x 座標: 395
寬度: 0.1051948048942486
時間:2.216666666666667
===
檔案名稱: frame_0133.jpg
最遠 x 座標: 508
寬度: 0.1051948048942486
時間:2.2333333333333334
===
檔案名稱: frame_0134.jpg
最遠 x 座標: 613
寬度: 0.2688311680630798
時間:2.25
===
檔案名稱: frame_0135.jpg
最遠 x 座標: 658
寬度: 0.3331168821651206
時間:2.2666666666666666
===
檔案名稱: frame_0136.jpg
最遠 x 座標: 690
寬度: 0.09935064906679035
時間:2.283333333333333
===
檔案名稱: frame_0137.jpg
最遠 x 座標: 710
寬度: 0.23961038892578848
時間:2.3
===
檔案名稱: frame_0138.jpg
最遠 x 座標: 655
寬度: 0.09350649323933209
時間:2.316666666666667
===
檔案名稱: frame_0139.jpg
最遠 x 座標: 607
寬度: 0.2103896097884972
時間:2.3333333333333335
===
檔案名稱: frame_0140.jpg
最遠 x 座標: 544
寬度: 0.11688311654916511
時間:2.35
===
檔案名稱: frame_0141.jpg
最遠 x 座標: 532
寬度: 0.30389610302782927
時間